# Project 5: PISA Data Wrangling
Within this notebook, each data wrangling step is carried out. This is essential to prepare the raw PISA 2012 dataset. The process begins with an initial assessment of data quality and tidiness, followed by a number of cleaning steps. As a final result, the cleaned data is stored for further analytical purposes.

In [1]:
# Import all packages and set plots to be embedded inline
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.options.mode.chained_assignment = None 
import numpy as np
import csv
import os

## Gather
<p>As a first step, the CSV file with the PISA data is read in. Then, the created dataframe is reduced to only the necessary parts.<p>

In [2]:
# Read in pisa data
pisa_data = pd.read_csv("pisa2012.csv", encoding="cp1252", sep=",")
pisa_dict = pd.read_csv('pisadict2012.csv', encoding="cp1252", sep=",", header=None, names=["A", "B"])
pisa_dict = dict(zip(pisa_dict["A"], pisa_dict["B"]))


# Reduce the entirety over 350 columns to 40 essential columns
pisa_columns = ["CNT", "OECD", "ST04Q01", "AGE", 
                "IC01Q01", "IC01Q04", "IC02Q01", "IC02Q04", "IC03Q01", "IC04Q01", 
                "IC10Q01", "IC10Q02", "IC10Q03", "IC10Q04", "IC10Q05", "ST57Q06",
            "PV1MATH", "PV2MATH", "PV3MATH", "PV4MATH", "PV5MATH",   
            "PV1READ", "PV2READ", "PV3READ", "PV4READ", "PV5READ",
            "PV1SCIE", "PV2SCIE", "PV3SCIE", "PV4SCIE", "PV5SCIE", "OUTHOURS", "TIMEINT"]
pisa_new = pisa_data[pisa_columns]

C:\Users\schne\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (15,16,17,21,22,23,24,25,26,30,31,36,37,45,65,123,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,475) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Assess
The second step involves the assessment of the data at hand. The PISA program data from 2012 are assessed visually as well as programmatically in consideration of quality and tidiness issues. A summary of the findings is provided below.

In [3]:
# Explore the dataframe visually as a first assessment
pisa_new.sample(10)

,CNT,OECD,ST04Q01,AGE,IC01Q01,IC01Q04,IC02Q01,IC02Q04,IC03Q01,IC04Q01,IC10Q01,IC10Q02,IC10Q03,IC10Q04,IC10Q05,ST57Q06,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH,PV1READ,PV2READ,PV3READ,PV4READ,PV5READ,PV1SCIE,PV2SCIE,PV3SCIE,PV4SCIE,PV5SCIE,OUTHOURS,TIMEINT
246708,Italy,OECD,Female,16.08,"Yes, and I use it","Yes, and I use it","Yes, and I use it","Yes, and I use it",6 years old or younger,10-12 years old,Once or twice a week,Never or hardly ever,Once or twice a month,Never or hardly ever,Once or twice a month,2.0,281.2821,312.4396,268.0402,317.8922,342.0393,385.6415,399.9391,388.8187,382.4643,392.7903,374.6177,401.6598,402.5923,412.8497,395.1324,5.0,81.0
455762,Thailand,Non-OECD,Female,15.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,283.3853,273.2591,327.0058,355.0475,364.3948,358.3967,398.1122,395.7292,395.7292,465.6285,331.1637,342.3536,402.9653,392.7079,430.0075,7.0,NaN
64306,Brazil,Non-OECD,Female,16.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,342.5845,397.1101,368.2895,395.5523,404.1206,413.2041,464.0399,449.7423,427.5016,446.5651,410.9847,431.4994,375.5502,463.2040,426.8370,3.0,NaN
241643,Italy,OECD,Male,15.75,No,No,"Yes, and I use it","Yes, and I use it",7-9 years old,10-12 years old,Never or hardly ever,Never or hardly ever,Never or hardly ever,Never or hardly ever,Never or hardly ever,0.0,450.9347,422.1140,440.0296,462.6188,455.6084,396.8644,380.0235,404.0819,424.1305,380.8255,455.7441,410.9847,438.0269,451.0817,430.5670,1.0,86.0
19089,Argentina,Non-OECD,Male,15.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,403.1080,378.1820,387.5292,368.8347,368.0558,457.0904,439.4476,432.2301,411.3796,464.3079,394.1999,407.2548,394.1999,369.0228,394.1999,NaN,NaN
461539,Tunisia,Non-OECD,Female,15.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355.3591,329.6542,327.3173,307.0650,273.5706,349.8181,319.6343,319.6343,280.7131,362.5271,341.7941,359.5114,338.0642,322.2119,340.8616,NaN,NaN
479081,United States of America,OECD,Male,16.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,446.8064,474.8481,463.9430,528.5948,431.2276,414.5072,444.1791,444.9811,425.7344,407.2896,407.3480,462.3648,469.8247,482.8795,423.2003,NaN,NaN
144821,Denmark,OECD,Male,15.58,No,"Yes, and I use it","Yes, but I don’t use it","Yes, and I use it",7-9 years old,7-9 years old,Almost every day,Never or hardly ever,Almost every day,Once or twice a month,Never or hardly ever,NaN,626.1178,635.4650,625.3388,616.7705,620.6652,574.9764,590.2133,580.5900,591.0153,590.2133,614.6401,614.6401,605.3152,622.1000,659.3995,NaN,32.0
160528,Spain,OECD,Female,15.67,"Yes, and I use it","Yes, and I use it","Yes, and I use it","Yes, and I use it",6 years old or younger,7-9 years old,Never or hardly ever,Never or hardly ever,Once or twice a week,Once or twice a month,Never or hardly ever,0.0,575.0195,552.4303,537.6305,515.0413,479.2102,581.5184,595.0217,548.9517,586.2842,548.9517,552.4431,577.6203,588.8101,610.2574,557.1056,13.0,39.0
162498,Spain,OECD,Female,15.50,"Yes, and I use it","Yes, and I use it","Yes, and I use it","Yes, and I use it",6 years old or younger,7-9 years old,Once or twice a week,Once or twice a week,Once or twice a week,Once or twice a week,Once or twice a month,NaN,431.0718,491.0500,459.8925,440.4191,474.6923,482.0708,497.1626,448.7097,511.4602,500.3399,451.2682,463.3905,461.5256,444.7408,473.6479,NaN,75.0


In [4]:
# Explore data programmatically by using built-in pandas functions
# Assess missing values and data types
pisa_new.info()

# Determine number of duplicate rows
print("Number of duplicate rows:", len(pisa_new[pisa_new.duplicated(keep='first')]))

# Assess numeric variables through summary statistics
pisa_new.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485490 entries, 0 to 485489
Data columns (total 33 columns):
CNT         485490 non-null object
OECD        485490 non-null object
ST04Q01     485490 non-null object
AGE         485374 non-null float64
IC01Q01     296977 non-null object
IC01Q04     297305 non-null object
IC02Q01     296975 non-null object
IC02Q04     296944 non-null object
IC03Q01     293216 non-null object
IC04Q01     296305 non-null object
IC10Q01     291811 non-null object
IC10Q02     291025 non-null object
IC10Q03     290262 non-null object
IC10Q04     290907 non-null object
IC10Q05     291025 non-null object
ST57Q06     289428 non-null float64
PV1MATH     485490 non-null float64
PV2MATH     485490 non-null float64
PV3MATH     485490 non-null float64
PV4MATH     485490 non-null float64
PV5MATH     485490 non-null float64
PV1READ     485490 non-null float64
PV2READ     485490 non-null float64
PV3READ     485490 non-null float64
PV4READ     485490 non-null float64
PV5

,AGE,ST57Q06,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH,PV1READ,PV2READ,PV3READ,PV4READ,PV5READ,PV1SCIE,PV2SCIE,PV3SCIE,PV4SCIE,PV5SCIE,OUTHOURS,TIMEINT
count,485374.000000,289428.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.000000,485490.00000,485490.000000,308799.000000,297074.000000
mean,15.784283,1.583081,469.621653,469.648358,469.648930,469.641832,469.695396,472.004640,472.068052,472.022059,471.926562,472.013506,475.769824,475.813674,475.851549,475.78524,475.820184,11.104100,50.895996
std,0.290221,2.760885,103.265391,103.382077,103.407631,103.392286,103.419170,102.505523,102.626198,102.640489,102.576066,102.659989,101.464426,101.514649,101.495072,101.51220,101.566347,10.476669,40.987895
min,15.170000,0.000000,19.792800,6.473000,42.226200,24.622200,37.085200,0.083400,0.703500,0.703500,4.134400,2.307400,2.648300,2.834800,11.879900,8.42970,17.754600,0.000000,0.000000
25%,15.580000,0.000000,395.318600,395.318600,395.240700,395.396500,395.240700,403.600700,403.360100,403.360100,403.354600,403.360100,404.457300,404.457300,404.550500,404.45730,404.457300,4.000000,19.000000
50%,15.750000,1.000000,466.201900,466.124000,466.201900,466.279800,466.435600,475.455000,475.535200,475.455000,475.535200,475.535200,475.699400,475.606100,475.699400,475.97910,475.885900,8.000000,39.000000
75%,16.000000,2.000000,541.057800,541.447300,541.291500,541.447300,541.447300,544.502500,544.503500,544.503500,544.502500,544.503500,547.780700,547.873900,547.967200,547.78070,547.780700,14.000000,71.000000
max,16.330000,30.000000,962.229300,957.010400,935.745400,943.456900,907.625800,904.802600,881.239200,884.447000,881.159000,901.608600,903.338300,900.540800,867.624000,926.55730,880.958600,180.000000,206.000000


### Summary
#### Quality
- some columns have multiple data types
- there are many missing entries for the information and communication (IC) columns
- potential outlier values for variable OUTHOURS (learning time out of school)

#### Tidiness
- test scores for READ, MATH and SCIENCE are split into five columns with plausible values
- column names should be renamed for reasons of consistency and clarity

## Clean
The third step is dedicated to data cleaning. Therefore, documented quality and tidiness issues are sequentially resolved.
### Tidiness
Taking into account the five different plausible test scores, average test scores are computed for READ, MATH, and SCIENCE. In addition, a total average score is calculated for each student record.

In [5]:
# Create average of MATH, READ and SCIENCE scores and store results in new columns
pisa_new["avg_math_score"] = (pisa_new["PV1MATH"] + pisa_new["PV2MATH"] + pisa_new["PV3MATH"] + pisa_new["PV4MATH"] + pisa_new["PV5MATH"]) / 5
pisa_new["avg_read_score"] = (pisa_new["PV1READ"] + pisa_new["PV2READ"] + pisa_new["PV3READ"] + pisa_new["PV4READ"] + pisa_new["PV5READ"]) / 5
pisa_new["avg_science_score"] = (pisa_new["PV1SCIE"] + pisa_new["PV2SCIE"] + pisa_new["PV3SCIE"] + pisa_new["PV4SCIE"] + pisa_new["PV5SCIE"]) / 5
pisa_new["avg_total_score"] = (pisa_new["avg_math_score"] + pisa_new["avg_read_score"] + pisa_new["avg_science_score"]) / 3

pisa_new.drop(columns=["PV1MATH", "PV2MATH", "PV3MATH", "PV4MATH", "PV5MATH",   
            "PV1READ", "PV2READ", "PV3READ", "PV4READ", "PV5READ", 
            "PV1SCIE", "PV2SCIE", "PV3SCIE", "PV4SCIE", "PV5SCIE"], inplace=True)

For ensuring more transparency, the column name codes are replaced by their dictionary definition.

In [6]:
# Rename columns of dataframe
pisa_new.rename({"CNT":"country", "ST04Q01":"gender", "IC01Q01":"at_home_computer", "IC01Q04":"at_home_internet", "AGE":"age", "OECD":"oecd",
                 "IC02Q01":"at_school_computer", "IC02Q04":"at_school_internet", "IC03Q01":"first_use_computer", "IC04Q01":"first_use_internet", 
                 "IC10Q01":"at_school_chatting", "IC10Q02":"at_school_emailing", "IC10Q03":"at_school_browsing", "IC10Q04":"at_school_downloading", "IC10Q05":"at_school_posting",
                 "OUTHOURS":"at_home_study_time", "TIMEINT":"at_school_computer_study_time", "ST57Q06":"at_home_computer_study_time"}, 
          axis="columns", inplace=True)

# Test if changes were successful
pisa_new.sample(5)

,country,oecd,gender,age,at_home_computer,at_home_internet,at_school_computer,at_school_internet,first_use_computer,first_use_internet,at_school_chatting,at_school_emailing,at_school_browsing,at_school_downloading,at_school_posting,at_home_computer_study_time,at_home_study_time,at_school_computer_study_time,avg_math_score,avg_read_score,avg_science_score,avg_total_score
25797,Australia,OECD,Male,15.50,"Yes, and I use it","Yes, and I use it","Yes, and I use it","Yes, and I use it",7-9 years old,7-9 years old,Never or hardly ever,Never or hardly ever,Almost every day,Almost every day,Once or twice a month,NaN,4.0,94.0,361.04532,354.92262,359.97762,358.648520
168545,Spain,OECD,Female,15.75,"Yes, and I use it","Yes, and I use it","Yes, and I use it","Yes, and I use it",10-12 years old,10-12 years old,Once or twice a month,Once or twice a week,Once or twice a week,Once or twice a week,Never or hardly ever,2.0,6.0,28.0,470.64186,454.50820,432.43194,452.527333
317692,Mexico,OECD,Male,15.42,"Yes, and I use it","Yes, and I use it","Yes, and I use it","Yes, and I use it",10-12 years old,10-12 years old,Never or hardly ever,Never or hardly ever,Never or hardly ever,Never or hardly ever,Never or hardly ever,1.0,6.0,28.0,441.97692,474.25206,451.82768,456.018887
63474,Brazil,Non-OECD,Female,16.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,9.0,NaN,478.74282,475.79572,415.27414,456.604227
374646,Peru,Non-OECD,Male,16.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.0,NaN,371.56100,400.95426,390.84298,387.786080


### Quality
As pandas gave a warning message about inconsistent data types after the CSV file was read in, the columns must be casted to adequate data types.

In [7]:
# Do conversion for string data types
for c in ["country", "gender", "at_home_computer", "at_home_internet", "at_school_computer",
          "at_school_internet", "first_use_computer", "first_use_internet", "at_school_chatting", 
          "at_school_emailing", "at_school_browsing", "at_school_downloading", "at_school_posting", "oecd"]:
    pisa_new[c] = pisa_new[c].astype(str)

# Do conversion for float data types
for c in ["age", "at_home_study_time", "at_home_computer_study_time", "at_school_computer_study_time", "avg_math_score", 
          "avg_read_score", "avg_science_score", "avg_total_score"]:
    pisa_new[c] = pisa_new[c].astype(float)

# Transform empty strings to real NaN values
pisa_new.replace(to_replace="None", value=np.nan, inplace=True)
pisa_new.replace(to_replace="nan", value=np.nan, inplace=True)

# Test if changes were successful
pisa_new.dtypes

country                           object
oecd                              object
gender                            object
age                              float64
at_home_computer                  object
at_home_internet                  object
at_school_computer                object
at_school_internet                object
first_use_computer                object
first_use_internet                object
at_school_chatting                object
at_school_emailing                object
at_school_browsing                object
at_school_downloading             object
at_school_posting                 object
at_home_computer_study_time      float64
at_home_study_time               float64
at_school_computer_study_time    float64
avg_math_score                   float64
avg_read_score                   float64
avg_science_score                float64
avg_total_score                  float64
dtype: object

Next, the large number of missing values for columns related to the information and communication technology (ICT) questionnaire needs to be addressed. If there is not a single answer for the ICT questions, the respective row is deleted.

In [8]:
# Drop rows if they do not contain at least one answer for the first couple of ICT questions
pisa_new.dropna(subset=["at_home_computer", "at_home_internet", "at_school_computer", "at_school_internet"], thresh=1, inplace=True)

# Test if changes were successful
pisa_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299843 entries, 22151 to 475552
Data columns (total 22 columns):
country                          299843 non-null object
oecd                             299843 non-null object
gender                           299843 non-null object
age                              299757 non-null float64
at_home_computer                 296977 non-null object
at_home_internet                 297305 non-null object
at_school_computer               296975 non-null object
at_school_internet               296944 non-null object
first_use_computer               292722 non-null object
first_use_internet               295779 non-null object
at_school_chatting               291223 non-null object
at_school_emailing               290458 non-null object
at_school_browsing               289690 non-null object
at_school_downloading            290362 non-null object
at_school_posting                290482 non-null object
at_home_computer_study_time      186720 non-

Considering the column for study time out of school per week, there is a large number of outliers. It seems to be unrealistic that a student learns more than 40 hours, i. e. a full-time job, besides the time spent at school. So, all  values that exceed this limit are removed.

In [9]:
# Remove study time values that are larger than 40
pisa_new.loc[pisa_new["at_home_study_time"] > 40, "at_home_study_time"] = np.nan

# Test if changes were successful
print("Maximum hours of study time out of school:", max(pisa_new.at_home_study_time))

Maximum hours of study time out of school: 40.0


In [10]:
# Store prepared dataframe in new CSV file
pisa_new.to_csv("pisa_new.csv", index=False, encoding="utf-8")